In [1]:
!pip install openai

     |████████████████████████████████| 152 kB 13.4 MB/s 
     |████████████████████████████████| 11.3 MB 66.5 MB/s 
     |████████████████████████████████| 161 kB 90.0 MB/s 
     |████████████████████████████████| 242 kB 75.6 MB/s 
  Created wheel for openai: filename=openai-0.11.4-py3-none-any.whl size=161960 sha256=38578d9839036cebe66d50521323817efaf04ee0f66cd583b2596642bdca7a87
  Stored in directory: /root/.cache/pip/wheels/f6/60/b2/2275a37f6383cec638f9181c7289f2284ba574f8ddc7a836b8
Successfully built openai
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 2.5.9
    Uninstalling openpyxl-2.5.9:
      Successfully uninstalled openpyxl-2.5.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency confli

In [ ]:
import os
import openai
import pandas as pd

In [ ]:
import json

with open('/content/openai_cofig.json', 'r') as f:
    config = json.load(f)

In [ ]:
openai.api_key = config.get('api_key')

## Preparing the Data 

In [ ]:
data = pd.read_excel('/content/semantic_baseline_records_all_V1.xlsx')
data.head()

FileNotFoundError: ignored

In [ ]:
credit_df = data[data['Q']=='Q2'].reset_index(drop=True)

In [ ]:
credit_df.columns

Index(['Q', 'Question', 'Company Name', 'url', 'html_text', 'found_noun',
       'context', 'context_verb', 'confidence', 'Status', 'New Context',
       'Avg Sim', 'Comments', 'id', 'answer', 'score', 'Unnamed: 6'],
      dtype='object')

In [ ]:
print('Question:' , credit_df['Question'][0])
credit_subdf = credit_df.loc[:,['Company Name','url','New Context','Status']]
credit_subdf['Status'] = credit_subdf['Status'].apply(lambda x: 'Yes' if x == 'TP' else 'No')
credit_subdf.head()

Question: Does the insured/applicant directly accept or process credit card payments (in-person or online) or process financial/banking transactions?


,Company Name,url,New Context,Status
0,SMOKING JS BBQ CATERING,http://smokingjsbbq.com/menus.html,We accept PayPal payments for your deposit or ...,Yes
1,ATLANTIC HOUSING FOUNDATION,http://www.atlantichousing.org/,We also want to remind you that we accept elec...,Yes
2,CEDAR CREEK WOODSHOP,https://cedarcreekwoodshop.com/contact.html,Accepts Personal Checks Money Orders Cashiers ...,Yes
3,CHAMPION DOOR SERVICE,https://champion-garage-door-service.com/,All of our master craftsmen are equipped with ...,Yes
4,MR HEADLIGHT RESTORATION CO,https://mrheadlightrc.com/,We Accept The Following Payments Plus Cash . I...,Yes


In [ ]:
texts = credit_df['New Context'].tolist()
labels = credit_subdf['Status'].tolist()
metadata = credit_subdf.to_dict('records')

creds_fileinput_df = pd.DataFrame({'text':texts,'label':labels,'metadata':metadata})

In [ ]:
creds_fileinput_df.shape

(30, 3)

In [ ]:
creds_fileinput_df.head()

,text,label,metadata
0,We accept PayPal payments for your deposit or ...,Yes,"{'Company Name': 'SMOKING JS BBQ CATERING', 'u..."
1,We also want to remind you that we accept elec...,Yes,{'Company Name': 'ATLANTIC HOUSING FOUNDATION'...
2,Accepts Personal Checks Money Orders Cashiers ...,Yes,"{'Company Name': 'CEDAR CREEK WOODSHOP', 'url'..."
3,All of our master craftsmen are equipped with ...,Yes,"{'Company Name': 'CHAMPION DOOR SERVICE', 'url..."
4,We Accept The Following Payments Plus Cash . I...,Yes,{'Company Name': 'MR HEADLIGHT RESTORATION CO'...


In [ ]:
creds_fileinput = creds_fileinput_df.to_dict('records')

In [ ]:
with open("cred_data.jsonl", 'w') as f:
    for item in creds_fileinput:
        f.write(json.dumps(item) + "\n")

with open("cred_data.jsonl") as f:
    for line in f:
        print(line)

{"text": "We accept PayPal payments for your deposit or full payment on your Party . Pay safely by major credit card . To Book us or obtain a Rate Quote . Smoking Js works closely with other local traditional caterers and can provide . for a price quote on a smaller party .", "label": "Yes", "metadata": {"Company Name": "SMOKING JS BBQ CATERING", "url": "http://smokingjsbbq.com/menus.html", "New Context": "We accept PayPal payments for your deposit or full payment on your Party . Pay safely by major credit card . To Book us or obtain a Rate Quote . Smoking Js works closely with other local traditional caterers and can provide . for a price quote on a smaller party .", "Status": "Yes"}}

{"text": "We also want to remind you that we accept electronic rental payments via Rent Caf and WIPS This simple and easy process allows you to make your payment while minimizing persontoperson contact . We will be waiving all online payment fees associated with these payment options This includes credi

## Aircraft Question

In [ ]:
data = pd.read_excel('/content/Output_new_Q1.xlsx')
data.head()

,Business Name,Address,Question,Expected Response,Snippets,Url,Machine result,Machine Snippet,Machine url,Validation,Metrics,UW Review,UW Response,UW Comments,webtext,New Context,Avg Sim,label,Score,Avg,Output
0,"AMERICAN PORTWELL TECHNOLOGY, INC.","44200 Christy Street, Fremont, CA 94538 (USA)",Does the applicant provide products or work re...,1,Aircraft communications and navigation systems...,https://portwell.com/solutions/military.php,YES,Aircraft communications and navigation systems...,https://portwell.com/solutions/military.php,Correct,TP,Yes,Agree,NaN,Military Embedded Computing Solutions \n Home ...,"navigation, command and control, and...",0.617131,We provide products or work related to the ope...,0.967396,0.634645,1
1,BAE Systems Inc,"2941 Fairview Park Dr, Falls Church, VA 22042",Does the applicant provide products or work re...,1,We’ve been improving flight controls for 40+ y...,https://www.baesystems.com/en-us/product/fligh...,YES,"Mission-critical flight, pilot, and engine con...",https://www.baesystems.com/en-us/our-company/i...,Correct,TP,Yes,Agree,NaN,Flight Control Systems: Ground Collision Avoid...,Our products include fly-by-wire flight contr...,0.551032,We provide products or work related to the ope...,0.934713,0.570216,1
2,PCB POWER INC,"18153 Napa Street, Northridge Estate, Californ...",Does the applicant provide products or work re...,1,"Automotive\tNavigation systems, music systems,...",https://usa.pcbpower.com/flexible-and-rigid-fl...,YES,"Automotive Navigation systems, music systems, ...",https://usa.pcbpower.com/flexible-and-rigid-fl...,Correct,TP,Yes,Agree,NaN,"RF PCB Manufacturer USA, RF PCB Suppliers USA,...","equipment, radio commu...",0.525229,We provide products or work related to the ope...,0.949932,0.546465,1
3,EMSG,"951 Monocacy Road, York, PA 17404",Does the applicant provide products or work re...,1,Marine navigation,https://emsginc.com/electronics-manufacturing-...,YES,With our ongoing commitment to quality control...,https://emsginc.com/industries-we-serve/traffi...,Correct,TP,Yes,Agree,Marine navigation,Electronics Manufacturing and PCB Assembly in ...,"Once strictly mechanical devices, vehicles now...",0.500635,We provide products or work related to the ope...,0.968354,0.524021,1
4,"GENERAL DYNAMICS MISSION SYSTEMS, INC","8220 East Roosevelt Street Scottsdale, AZ 85257",Does the applicant provide products or work re...,1,SPACE OPERATIONS & ENGINEERING SERVICES,https://gdmissionsystems.com/space,YES,Space Operations & Engineering Services,https://gdmissionsystems.com/products/communic...,Correct,TP,Yes,Agree,"Better snippet: We design, build and manage gr...",Space - General Dynamics Mission Systems \n La...,We build products and deliver technology for ...,0.498263,We provide products or work related to the ope...,0.955822,0.521141,1


In [ ]:
data.columns

Index(['Business Name', 'Address', 'Question', 'Expected Response', 'Snippets',
       'Url', 'Machine result', 'Machine Snippet', 'Machine url', 'Validation',
       'Metrics', 'UW Review', 'UW Response', 'UW Comments', 'webtext',
       'New Context', 'Avg Sim', 'label', 'Score', 'Avg', 'Output'],
      dtype='object')

In [ ]:
print('Question:' , data['Question'][0])
data_subdf = data.loc[:,['Business Name','Url','New Context','Metrics','Avg Sim', 'Score', 'Avg']]
data_subdf['Metrics'] = data_subdf['Metrics'].apply(lambda x: 'Yes' if x == 'TP' else 'No')
data_subdf.head()

Question: Does the applicant provide products or work related to the operation, safety, navigation, or control of any of the following: aircraft, spacecraft, watercraft, rail transport, automobiles, or motorcycles?


,Business Name,Url,New Context,Metrics,Avg Sim,Score,Avg
0,"AMERICAN PORTWELL TECHNOLOGY, INC.",https://portwell.com/solutions/military.php,"navigation, command and control, and...",Yes,0.617131,0.967396,0.634645
1,BAE Systems Inc,https://www.baesystems.com/en-us/product/fligh...,Our products include fly-by-wire flight contr...,Yes,0.551032,0.934713,0.570216
2,PCB POWER INC,https://usa.pcbpower.com/flexible-and-rigid-fl...,"equipment, radio commu...",Yes,0.525229,0.949932,0.546465
3,EMSG,https://emsginc.com/electronics-manufacturing-...,"Once strictly mechanical devices, vehicles now...",Yes,0.500635,0.968354,0.524021
4,"GENERAL DYNAMICS MISSION SYSTEMS, INC",https://gdmissionsystems.com/space,We build products and deliver technology for ...,Yes,0.498263,0.955822,0.521141


In [ ]:
for index,row in data_subdf.iterrows():
    text = row['New Context'].replace("  "," ")
    row['New Context'] = text

In [ ]:
data_subdf.head()

,Business Name,Url,New Context,Metrics,Avg Sim,Score,Avg
0,"AMERICAN PORTWELL TECHNOLOGY, INC.",https://portwell.com/solutions/military.php,"navigation, command and control, and...",Yes,0.617131,0.967396,0.634645
1,BAE Systems Inc,https://www.baesystems.com/en-us/product/fligh...,Our products include fly-by-wire flight contr...,Yes,0.551032,0.934713,0.570216
2,PCB POWER INC,https://usa.pcbpower.com/flexible-and-rigid-fl...,"equipment, radio commu...",Yes,0.525229,0.949932,0.546465
3,EMSG,https://emsginc.com/electronics-manufacturing-...,"Once strictly mechanical devices, vehicles now...",Yes,0.500635,0.968354,0.524021
4,"GENERAL DYNAMICS MISSION SYSTEMS, INC",https://gdmissionsystems.com/space,We build products and deliver technology for ...,Yes,0.498263,0.955822,0.521141


## Engine list

#### To get List of Engines Available for your service

In [ ]:
openai.Engine.list()

<OpenAIObject list at 0x7fa0bcb03650> JSON: {
  "data": [
    {
      "created": null,
      "id": "ada",
      "max_replicas": null,
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true,
      "ready_replicas": null,
      "replicas": null
    },
    {
      "created": null,
      "id": "ada-code-search-code",
      "max_replicas": null,
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true,
      "ready_replicas": null,
      "replicas": null
    },
    {
      "created": null,
      "id": "ada-code-search-text",
      "max_replicas": null,
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true,
      "ready_replicas": null,
      "replicas": null
    },
    {
      "created": null,
      "id": "ada-instruct-beta",
      "max_replicas": null,
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true,
      "ready_r

## File's Operations

<h5> Upload Files </h5>

In [ ]:
openai.File.create(
  file=open("/content/cred_data.jsonl"),
  purpose='classifications'
)

<File file id=file-bpNOGfmNKKUyqpGo0FDHohLx at 0x7fa0bcb03830> JSON: {
  "bytes": 35389,
  "created_at": 1639152513,
  "filename": "cred_data.jsonl",
  "id": "file-bpNOGfmNKKUyqpGo0FDHohLx",
  "object": "file",
  "purpose": "classifications",
  "status": "uploaded",
  "status_details": null
}

<h5> File List </h5>

In [ ]:
openai.File.list()

<OpenAIObject list at 0x7fa0bcaf5cb0> JSON: {
  "data": [
    {
      "bytes": 35389,
      "created_at": 1639152513,
      "filename": "cred_data.jsonl",
      "id": "file-bpNOGfmNKKUyqpGo0FDHohLx",
      "object": "file",
      "purpose": "classifications",
      "status": "processed",
      "status_details": null
    }
  ],
  "object": "list"
}

<h5> Take the file_id to be used for QA Classification task  </h5>

## Zero Shot Classification

## To get Predicted Label Probabilities

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.3 MB 8.0 MB/s 
     |████████████████████████████████| 61 kB 408 kB/s 
     |████████████████████████████████| 3.3 MB 44.0 MB/s 
     |████████████████████████████████| 596 kB 27.3 MB/s 
     |████████████████████████████████| 895 kB 72.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import GPT2TokenizerFast

# Load the tokenizer.
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# Make sure the labels are formatted correctly.
labels = ["Yes", "No", "Maybe"]
labels = [label.strip().lower().capitalize() for label in labels]

# Encode the labels with extra white space prepended.
labels_tokens = {label: tokenizer.encode(" " + label) for label in labels}
print(labels_tokens)

"""Output:
{'Positive': [33733], 'Negative': [36183]}
"""

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

{'Yes': [3363], 'No': [1400], 'Maybe': [6674]}


"Output:\n{'Positive': [33733], 'Negative': [36183]}\n"

In [ ]:
classification_response = openai.Classification.create(
  search_model="ada",
  model="curie",
  file = "file-bpNOGfmNKKUyqpGo0FDHohLx",
#   examples=[
#             ["will be waiving all online payment fees associated with these payment options This includes credit  card payments  echeck WIPS etc Please contact your leasing office with any questions or if you would like","Yes"],
#             ["We do not have the Payment Methods Accepted at this place Most places accept Cash and Major Credit Cards Please contact Summerfield SuitesDallas at to find out payment options Other details . Please call for check in and check out times . All rights reserved Information deemed reliable but not guaranteed . Send Summerfield SuitesDallas to a Phone . To lookup step by step driving directions to Summerfield SuitesDallas please enter your starting Address .","No"],
#            # ["We do not have the Payment Methods Accepted at this Pizza Shop Most Business accept Cash and Major Credit Cards Please contact Marios at to find out payment options Other details . Please call for current hours of operation . All rights reserved Information deemed reliable but not guaranteed . This is a Pizza shop located in Erie PA This is the only Marios location in Erie . To lookup step by step driving directions to Marios please enter your starting Address .","No"],
#            # ["We accept cash credit card payments CareCredit and LendingUSA . Please contact us for insurance eligibility and verification . She earned her medical degree from Dayanand Medical College and Hospital in India She completed her internship and residency at Arrowhead Regional Medical Center in California . Visit now or book an appointment . Dr BambrahDhamija has access to the best hospitals specialists labs radiology facilities and support programs available .","Yes"],
#             ["for local services shopping travel and Groupon updates . Buying is fast easy and secure . Groupon has verified that the customer actually visited Tranquility Well Spa . Sign in to get personalized notifications about your deals cash back special offers and more . Do Not Sell My Personal Information .","Yes"]
#             ],
  query="Does the insured/applicant directly accept or process credit card payments in-person or online?",
  labels=labels,
  max_examples = 10, # Needed for filtering search as inputs
  return_prompt = True , # Used for Debugginf purpose,
  return_metadata = True , # as we are inputing custom dataset 
  expand=["completion"],
  logprobs=len(labels)+1, 
  
)

In [ ]:
classification_response

<OpenAIObject classification at 0x7f9f48c9ab30> JSON: {
  "completion": {
    "choices": [
      {
        "finish_reason": "length",
        "index": 0,
        "logprobs": {
          "text_offset": [
            6187,
            6191
          ],
          "token_logprobs": [
            -0.19785161,
            -0.027932711
          ],
          "tokens": [
            " Yes",
            "\n"
          ],
          "top_logprobs": [
            {
              " Maybe": -3.8401399,
              " No": -2.2647576,
              " Online": -5.575163,
              " Yes": -0.19785161
            },
            {
              "\n": -0.027932711,
              " .": -5.58452,
              ",": -5.392694,
              "<|endoftext|>": -4.890224
            }
          ]
        },
        "text": " Yes\n"
      }
    ],
    "created": 1639156723,
    "id": "cmpl-4Ddpbzf47U8QBpKU6sFoi7pARyygB",
    "model": "curie:2020-05-03",
    "object": "text_completion"
  },
  "file": "file-b

## Didn't Understand so far.

In [ ]:
import numpy as np

# Take the starting tokens for probability estimation.
# Labels should have distinct starting tokens.
# Here tokens are case-sensitive.
first_token_to_label = {tokens[0]: label for label, tokens in labels_tokens.items()}

top_logprobs = classification_response["completion"]["choices"][0]["logprobs"]["top_logprobs"][0]
token_probs = {
    tokenizer.encode(token)[0]: np.exp(logp) 
    for token, logp in top_logprobs.items()
}
label_probs = {
    first_token_to_label[token]: prob 
    for token, prob in token_probs.items()
    if token in first_token_to_label
}

# Fill in the probability for the special "Unknown" label.
if sum(label_probs.values()) < 1.0:
    label_probs["Unknown"] = 1.0 - sum(label_probs.values())

print(label_probs)
# """Output:
# {'Negative': 0.053965452806285695,
#  'Positive': 0.901394752718519,
#  'Unknown': 0.04463979447519528}
# """

{'Maybe': 0.02149059460058842, 'No': 0.10385520599182627, 'Yes': 0.8204915968528896, 'Unknown': 0.05416260255469574}


In [ ]:
with open("Credit_Card_Classification_Output.jsonl", 'w') as f:
    #for item in classification_response:
    f.write(json.dumps(classification_response) )


In [ ]:
for item in classification_response['selected_examples']

## Question Answer Module

#### Best if you want give a single context , and you have multiple Question Answer pairs to be given in the prompt . i.e (Examples Parameter)

In [ ]:
openai.Answer.create(
  #search_model="ada",
  model="curie",
  question="Accept or process credit card payments in-person or online ?",
  documents=["get them done . Request a Quote . Name . Email . INFORMATION SECTION WE ACCEPT CASH CASH APP OR  CARD PAYMENTS  LEAVE THE FOLLOWING INFO IN THE MESSAGE BOX WHAT IS THE ADDRESS YOU NEED SERVICE IN WHAT IS THE"],
  
  # EXAMPLE CONTEXT is TEXT , should be single
  examples_context="Does the insured/applicant directly accept or process credit card payments in-person or online ?",
  
  # Best to give 2-3 examples per usecase

  # THIS IS WORNG - EXAMPLES Should be QUESTION and ANSWER PAIR
  examples=[["will be waiving all online payment fees associated with these payment options This includes credit  card payments  echeck WIPS etc Please contact your leasing office with any questions or if you would like","Yes"],
            ["We do not have the Payment Methods Accepted at this place Most places accept Cash and Major Credit Cards Please contact Summerfield SuitesDallas at to find out payment options Other details . Please call for check in and check out times . All rights reserved Information deemed reliable but not guaranteed . Send Summerfield SuitesDallas to a Phone . To lookup step by step driving directions to Summerfield SuitesDallas please enter your starting Address .","No"],
           # ["We do not have the Payment Methods Accepted at this Pizza Shop Most Business accept Cash and Major Credit Cards Please contact Marios at to find out payment options Other details . Please call for current hours of operation . All rights reserved Information deemed reliable but not guaranteed . This is a Pizza shop located in Erie PA This is the only Marios location in Erie . To lookup step by step driving directions to Marios please enter your starting Address .","No"],
           # ["We accept cash credit card payments CareCredit and LendingUSA . Please contact us for insurance eligibility and verification . She earned her medical degree from Dayanand Medical College and Hospital in India She completed her internship and residency at Arrowhead Regional Medical Center in California . Visit now or book an appointment . Dr BambrahDhamija has access to the best hospitals specialists labs radiology facilities and support programs available .","Yes"],
            ["for local services shopping travel and Groupon updates . Buying is fast easy and secure . Groupon has verified that the customer actually visited Tranquility Well Spa . Sign in to get personalized notifications about your deals cash back special offers and more . Do Not Sell My Personal Information .","Yes"]
            ],
  max_tokens=5,
  stop=["\n", "<|endoftext|>"],

)

<OpenAIObject answer at 0x7fa0bd5004d0> JSON: {
  "answers": [
    "No"
  ],
  "completion": "cmpl-4DXVKTwjLJLU1Speug5xMMgaF63OF",
  "model": "curie:2020-05-03",
  "object": "answer",
  "search_model": "ada",
  "selected_documents": [
    {
      "document": 0,
      "text": "get them done . Request a Quote . Name . Email . INFORMATION SECTION WE ACCEPT CASH CASH APP OR  CARD PAYMENTS  LEAVE THE FOLLOWING INFO IN THE MESSAGE BOX WHAT IS THE ADDRESS YOU NEED SERVICE IN WHAT IS THE "
    }
  ]
}

In [1]:
logit_bias = {str(first_token): 100 for first_token in first_token_to_label}


NameError: ignored